In [1]:
import pandas as pd
import numpy as np
import feather
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing as pe
from tqdm import tqdm

In [2]:
train_stage1 = pd.read_csv('../data/training_variants')
test_stage1 = pd.read_csv('../data/test_variants')
trainx_stage1 = pd.read_csv('../data/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
testx_stage1 = pd.read_csv('../data/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

In [3]:
test_stage1_soln = pd.read_csv('../data/stage1_solution_filtered.csv')

In [4]:
test_stage1_soln.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,12,1,0,0,0,0,0,0,0,0
1,19,0,1,0,0,0,0,0,0,0
2,21,0,1,0,0,0,0,0,0,0
3,55,0,0,0,1,0,0,0,0,0
4,64,0,0,0,1,0,0,0,0,0


In [5]:
stage1_soln_ids = test_stage1_soln['ID']

In [6]:
test_stage1_soln = test_stage1_soln.drop('ID',axis=1)

In [7]:
test_stage1_soln.shape

(368, 9)

In [8]:
test_stage1_soln['class'] = test_stage1_soln.idxmax(axis=1)

In [9]:
test_stage1_soln['class'] = test_stage1_soln['class'].map(lambda x: int(x[-1]))

In [10]:
test_stage1_soln['ID'] = stage1_soln_ids

In [11]:
# test_s1_sol['class'] = test_s1_sol.idxmax(axis=1)
# test_s1_sol['class'] = test_s1_sol['class'].str[-1].astype(int)
# test_s1_sol['class'] = test_s1_sol['class']

In [12]:
test_stage1_soln.tail()

,class1,class2,class3,class4,class5,class6,class7,class8,class9,class,ID
363,0,0,0,1,0,0,0,0,0,4,5600
364,0,0,0,1,0,0,0,0,0,4,5609
365,0,0,0,0,0,1,0,0,0,6,5626
366,0,0,0,1,0,0,0,0,0,4,5628
367,1,0,0,0,0,0,0,0,0,1,5650


In [13]:
test_stage1_soln.head()

,class1,class2,class3,class4,class5,class6,class7,class8,class9,class,ID
0,1,0,0,0,0,0,0,0,0,1,12
1,0,1,0,0,0,0,0,0,0,2,19
2,0,1,0,0,0,0,0,0,0,2,21
3,0,0,0,1,0,0,0,0,0,4,55
4,0,0,0,1,0,0,0,0,0,4,64


In [14]:
train_stage1.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [15]:
train_stage1 = pd.merge(train_stage1, trainx_stage1, how='left', on='ID').fillna('')


test_stage1 = pd.merge(test_stage1, testx_stage1, how='left', on='ID').fillna('')
#pid = test['ID'].values

In [16]:
train_stage1.head()

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [17]:
test_stage1.head()

,ID,Gene,Variation,Text
0,0,ACSL4,R570S,2. This mutation resulted in a myeloproliferat...
1,1,NAGLU,P521L,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,PAH,L333F,Vascular endothelial growth factor receptor (V...
3,3,ING1,A148D,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,TMEM216,G77A,Abstract Retinoblastoma is a pediatric retina...


In [18]:
train_stage1.shape

(3321, 5)

In [19]:
test_stage1.shape

(5668, 4)

In [20]:
train_stage1.tail()

,ID,Gene,Variation,Class,Text
3316,3316,RUNX1,D171N,4,Introduction Myelodysplastic syndromes (MDS) ...
3317,3317,RUNX1,A122*,1,Introduction Myelodysplastic syndromes (MDS) ...
3318,3318,RUNX1,Fusions,1,The Runt-related transcription factor 1 gene (...
3319,3319,RUNX1,R80C,4,The RUNX1/AML1 gene is the most frequent targe...
3320,3320,RUNX1,K83E,4,The most frequent mutations associated with le...


In [21]:
new_test = test_stage1[test_stage1.ID.isin(stage1_soln_ids)]

In [22]:
new_test.head()

,ID,Gene,Variation,Text
12,12,TET2,Y1902A,TET proteins oxidize 5-methylcytosine (5mC) on...
19,19,MTOR,D2512H,Genes encoding components of the PI3K-Akt-mTOR...
21,21,KIT,D52N,Myeloproliferative disorders (MPD) constitute ...
55,55,SPOP,F125V,"In the largest E3 ligase subfamily, Cul3 binds..."
64,64,KEAP1,C23Y,Keap1 is the substrate recognition module of a...


In [23]:
new_test = new_test.drop('ID',axis=1)

In [24]:
new_test_ids = [i for i in range(3321, 3321+new_test.shape[0])]

In [25]:
len(new_test_ids)

368

In [26]:
new_test['ID'] = new_test_ids

In [27]:
test_stage1_soln.head()

,class1,class2,class3,class4,class5,class6,class7,class8,class9,class,ID
0,1,0,0,0,0,0,0,0,0,1,12
1,0,1,0,0,0,0,0,0,0,2,19
2,0,1,0,0,0,0,0,0,0,2,21
3,0,0,0,1,0,0,0,0,0,4,55
4,0,0,0,1,0,0,0,0,0,4,64


In [28]:
new_test.shape

(368, 4)

In [29]:
test_stage1_soln.shape

(368, 11)

In [30]:
new_test['Class'] = test_stage1_soln['class'].values

In [31]:
new_test.shape

(368, 5)

In [32]:
new_test.head()

,Gene,Variation,Text,ID,Class
12,TET2,Y1902A,TET proteins oxidize 5-methylcytosine (5mC) on...,3321,1
19,MTOR,D2512H,Genes encoding components of the PI3K-Akt-mTOR...,3322,2
21,KIT,D52N,Myeloproliferative disorders (MPD) constitute ...,3323,2
55,SPOP,F125V,"In the largest E3 ligase subfamily, Cul3 binds...",3324,4
64,KEAP1,C23Y,Keap1 is the substrate recognition module of a...,3325,4


In [33]:
new_test.tail()

,Gene,Variation,Text,ID,Class
5600,MAP2K4,S251N,The systematic characterization of somatic mut...,3684,4
5609,VHL,N90I,The von Hippel-Lindau tumor suppressor protein...,3685,4
5626,MAP2K1,H119P,Genetic alterations that activate the mitogen-...,3686,6
5628,ATM,V2424G,Abstract The functional consequences of misse...,3687,4
5650,TP53,G245C,The transcription factor and tumor suppressor ...,3688,1


In [34]:
train_stage1.head()

,ID,Gene,Variation,Class,Text
0,0,FAM58A,Truncating Mutations,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,4,Oncogenic mutations in the monomeric Casitas B...


In [35]:
train_stage1.Class.value_counts()

7    953
4    686
1    568
2    452
6    275
5    242
3     89
9     37
8     19
Name: Class, dtype: int64

In [36]:
train_stage1.shape

(3321, 5)

In [37]:
train_stage2 = pd.concat([train_stage1, new_test], axis=0)

In [38]:
train_stage2.head()

,Class,Gene,ID,Text,Variation
0,1,FAM58A,0,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations
1,2,CBL,1,Abstract Background Non-small cell lung canc...,W802*
2,2,CBL,2,Abstract Background Non-small cell lung canc...,Q249E
3,3,CBL,3,Recent evidence has demonstrated that acquired...,N454D
4,4,CBL,4,Oncogenic mutations in the monomeric Casitas B...,L399V


In [39]:
train_stage2.tail()

,Class,Gene,ID,Text,Variation
5600,4,MAP2K4,3684,The systematic characterization of somatic mut...,S251N
5609,4,VHL,3685,The von Hippel-Lindau tumor suppressor protein...,N90I
5626,6,MAP2K1,3686,Genetic alterations that activate the mitogen-...,H119P
5628,4,ATM,3687,Abstract The functional consequences of misse...,V2424G
5650,1,TP53,3688,The transcription factor and tumor suppressor ...,G245C


In [40]:
len(train_stage2.ID.unique())

3689

In [41]:
train_stage2.shape

(3689, 5)

In [42]:
train_stage2.Class.value_counts()

7    1054
4     751
1     662
2     498
6     297
5     267
3      96
9      43
8      21
Name: Class, dtype: int64

In [43]:
len(train_stage2.ID.unique())

3689

In [44]:
feather.write_dataframe(train_stage2, '../cache/train_stage2.feather')

In [45]:
train_stage2 = feather.read_dataframe('../cache/train_stage2.feather')

In [46]:
train_stage2.Class.value_counts()

7    1054
4     751
1     662
2     498
6     297
5     267
3      96
9      43
8      21
Name: Class, dtype: int64

In [47]:
y_class = train_stage2.Class.values

In [48]:
# from imblearn.over_sampling import RandomOverSampler

In [49]:
train_stage2.ID[train_stage2['Class'] == 8]

121      121
220      220
306      306
307      307
308      308
750      750
1344    1344
1352    1352
1756    1756
1768    1768
1770    1770
1825    1825
2274    2274
2275    2275
2277    2277
2278    2278
2385    2385
3109    3109
3114    3114
3514    3514
3548    3548
Name: ID, dtype: int64

In [50]:
train_stage2[train_stage2.ID == 121]

,Class,Gene,ID,Text,Variation
121,8,SF3B1,121,The RNA maturation is an important and complex...,K700R


In [51]:
# sampler = RandomOverSampler()
# X_res, y_res = sampler.fit_sample(ts_2, y_class)

In [52]:
train_stage2 = train_stage2.drop('ID', axis=1)

In [53]:
max_rows = 1054 # chosen because class 7 has 955 rows in stage2 train set

In [54]:
s = train_stage2.Class.value_counts()

In [55]:
num_classes = 9
rows_to_append = []
ids = {}
for i in tqdm(range(1, num_classes+1)):
    class_rows = max_rows - s[i]
    rows_to_append.append(class_rows)
    ids[i] = train_stage2.index[train_stage2['Class'] == i]

100%|██████████| 9/9 [00:00<00:00, 2854.99it/s]


In [56]:
rows_to_append

[392, 556, 958, 303, 787, 757, 0, 1033, 1011]

In [57]:
import math

In [58]:
for i in tqdm(range(len(rows_to_append))):
    if rows_to_append[i] > 0:
        to_append = train_stage2.index.isin(ids[i+1])
#         print(to_append)
        n = to_append[to_append==True].sum()
        if n > rows_to_append[i]:
            m = rows_to_append[i]
            df = train_stage2.loc[to_append].copy()
            df = df[0:m-1]
            train_stage2 = pd.concat([train_stage2, df],axis=0)
        else:
            m = math.floor(rows_to_append[i]/n)
            for j in range(m):
                df = train_stage2.loc[to_append].copy()
                train_stage2 = pd.concat([train_stage2, df],axis=0)
            
        #df_to_append = train_stage2.loc[to_append].copy()
        #train_stage2 = pd.concat([train_stage2,df_to_append],axis=0)

100%|██████████| 9/9 [00:00<00:00, 55.34it/s]


In [59]:
train_stage2.Class.value_counts()

7    1054
4    1053
1    1053
8    1050
9    1032
2     996
3     960
6     891
5     801
Name: Class, dtype: int64

In [60]:
train_stage2.shape

(8890, 4)

In [61]:
train_stage2.head()

,Class,Gene,Text,Variation
0,1,FAM58A,Cyclin-dependent kinases (CDKs) regulate a var...,Truncating Mutations
1,2,CBL,Abstract Background Non-small cell lung canc...,W802*
2,2,CBL,Abstract Background Non-small cell lung canc...,Q249E
3,3,CBL,Recent evidence has demonstrated that acquired...,N454D
4,4,CBL,Oncogenic mutations in the monomeric Casitas B...,L399V


In [62]:
train_stage2.Class.value_counts()

7    1054
4    1053
1    1053
8    1050
9    1032
2     996
3     960
6     891
5     801
Name: Class, dtype: int64

In [63]:
feather.write_dataframe(train_stage2, '../cache/train_stage2_os.feather')

In [64]:
test_stage2 = pd.read_csv('../data/stage2_test_variants.csv')
testx_stage2 = pd.read_csv('../data/stage2_test_text.csv', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

In [65]:
test_stage2.head()

,ID,Gene,Variation
0,1,CHEK2,H371Y
1,2,AXIN2,Truncating Mutations
2,3,WNT4,E216G
3,4,SUCLA2,G118R
4,5,BRAF,T599insTT


In [66]:
testx_stage2.head()

,ID,Text
0,1,The incidence of breast cancer is increasing i...
1,2,An unselected series of 310 colorectal carcino...
2,3,Mycosis fungoides and Sézary syndrome are prim...
3,4,Regulated progression through the cell cycle ...
4,5,Pilocytic astrocytoma (PA) is emerging as a tu...


In [67]:
test = pd.merge(test_stage2, testx_stage2, how='left', on='ID').fillna('')

In [68]:
test.head()

,ID,Gene,Variation,Text
0,1,CHEK2,H371Y,The incidence of breast cancer is increasing i...
1,2,AXIN2,Truncating Mutations,An unselected series of 310 colorectal carcino...
2,3,WNT4,E216G,Mycosis fungoides and Sézary syndrome are prim...
3,4,SUCLA2,G118R,Regulated progression through the cell cycle ...
4,5,BRAF,T599insTT,Pilocytic astrocytoma (PA) is emerging as a tu...


In [69]:
test.shape

(986, 4)

In [70]:
len(set(test.Gene) -set(train_stage2.Gene))

132

In [71]:
len(set(test.Variation) -set(train_stage2.Variation))

612

In [72]:
feather.write_dataframe(test, '../cache/test_stage2.feather')